# This notebook implements oversampling to be included in the DRD Deep learning script

In [1]:
"""Routine for decoding the CIFAR-10 binary file format."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import argparse
import tensorflow as tf

# Process images of this size. Note that this differs from the original CIFAR
# image size of 32 x 32. If one alters this number, then the entire model
# architecture will change and any model would need to be retrained.
IMAGE_SIZE = 224

# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 5
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 2000 # was set from # 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 3500
batch_size = 100
data_dir = '/home/olle/PycharmProjects/Diabetic_Retinopathy_Detection/data/train'

In [ ]:
# read in an image to the class object

In [2]:
filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
             for i in xrange(0, 7)]
for f in filenames:
    if not tf.gfile.Exists(f):
        raise ValueError('Failed to find file: ' + f)
num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN

# Create a queue that produces the filenames to read.
filename_queue = tf.train.string_input_producer(filenames)

class SVHNRecord(object):
    pass

result = SVHNRecord()

# Dimensions of the images in the SVHN dataset.
# See http://ufldl.stanford.edu/housenumbers/ for a description of the
# input format.
result.height = 256
result.width = 256
result.depth = 3

reader = tf.TFRecordReader()
result.key, value = reader.read(filename_queue)
value = tf.parse_single_example(
    value,
    # Defaults are not specified since both keys are required.
    features={
        'image_raw': tf.FixedLenFeature(shape=[], dtype=tf.string),
        'label': tf.FixedLenFeature(shape=[], dtype=tf.int64),
        'image_name': tf.FixedLenFeature(shape=[], dtype=tf.string)
    })

# Convert from a string to a vector of uint8 that is record_bytes long.
record_bytes = tf.decode_raw(value['image_raw'], tf.uint8)
# record_bytes.set_shape([32*32*3])
record_bytes = tf.reshape(record_bytes, [256, 256, 3])
print("record bytes::::: ", record_bytes)
# Store our label to result.label and convert to int32
result.label = tf.cast(value['label'], tf.int32)
result.uint8image = record_bytes

record bytes:::::  Tensor("Reshape:0", shape=(256, 256, 3), dtype=uint8)


In [4]:
# Read examples from files in the filename queue.
reshaped_image = tf.cast(result.uint8image, tf.float32)

height = IMAGE_SIZE
width = IMAGE_SIZE

# Image processing for evaluation.
# Crop the central [height, width] of the image.
resized_image = tf.image.resize_image_with_crop_or_pad(reshaped_image,
                                                       height, width)

# Subtract off the mean and divide by the variance of the pixels.
float_image = tf.divide(resized_image, 255)

# Set the shapes of tensors.
float_image.set_shape([height, width, 3])
# read_input.label.set_shape([1])

# Ensure that the random shuffling has good mixing properties.
min_fraction_of_examples_in_queue = 0.4
min_queue_examples = int(num_examples_per_epoch *
                         min_fraction_of_examples_in_queue)

In [99]:
def _generate_image_and_label_batch(images_and_labels, min_queue_examples,
                                    batch_size, shuffle):

    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 16
    images, label_batch = tf.train.batch(
        [image, label],
        batch_size=batch_size,
        num_threads=num_preprocess_threads,
        capacity=min_queue_examples + 3 * batch_size)

    # Display the training images in the visualizer.
    tf.summary.image('images', images)

    return images, tf.reshape(label_batch, [batch_size])

In [101]:
images, labels = _generate_image_and_label_batch(images_and_labels,
                                           min_queue_examples, batch_size,
                                           shuffle=False)

In [90]:
images_and_labels=[]

In [114]:
# Convert 3D tensor of shape [height, width, channels] to 
# a 4D tensor of shape [batch_size, height, width, channels]
image = tf.expand_dims(float_image, 0)
label = tf.expand_dims(result.label, 0)
# Define the boolean predicate to be true when the class label is 1
pred = tf.equal(result.label, tf.convert_to_tensor([1]))
pred = tf.reshape(pred, [])

oversample_factor = 2
r_image = tf.cond(pred, lambda: tf.concat([image]*oversample_factor,0), lambda: image)
r_label = tf.cond(pred, lambda: tf.stack([label]*oversample_factor, 0), lambda: label)

images_and_labels.append([r_image, r_label])

num_preprocess_threads = 16

images, label_batch = tf.train.shuffle_batch_join(
    images_and_labels,
    batch_size=batch_size,
    capacity=2 * num_preprocess_threads * batch_size,
    min_after_dequeue=1 * num_preprocess_threads * batch_size,
    enqueue_many=True)

ValueError: Cannot infer Tensor's rank: Tensor("cond_22/Merge:0", dtype=int32)

In [95]:
label

<tf.Tensor 'ExpandDims_31:0' shape=(1,) dtype=int32>

# Built graph to execute data loading

In [16]:
# # Build an initialization operation to run below.
init = tf.global_variables_initializer()
# Start running operations on the Graph.
sess = tf.Session()
# sess.run(init)

# Start the queue runners.
tf.train.start_queue_runners(sess=sess)

sess.run(init)

In [109]:
sess.run(images)

ValueError: Operation u'batch_2' has been marked as not fetchable.

In [ ]:
class_weights =  1/prop
class_weights

In [ ]:
def undersampling():
    
    np.argwhere(prop==np.max(prop, axis=0))
    value_to_be_removed = values[prop==np.max(prop, axis=0)]
    index_to_delete = np.where(cls_true==value_to_be_removed)[0][0]
    new_cls = np.delete(cls_true, index_to_delete)
    
    return(new_cls)
    

In [ ]:
data_np.shape
value['label']

In [120]:
import numpy as np
import tensorflow as tf

class_num=8
data_np = np.random.choice(class_num,20000,p=[1.0/class_num]*class_num)

def sample(_):
    xx = tf.cast(tf.random_uniform([1])*class_num, tf.int32)[0]
    return xx

def fix_sample_and_rebalance(which='works'): 
    data_tensors = tf.constant(data_np, dtype=tf.int32)
    #data_tensors = images
    dataset = tf.data.Dataset.from_tensor_slices(data_tensors)

    target_dist = [1.0/class_num] * class_num
    target_dist[1]+=target_dist[0] ; target_dist[0]=0
    print('target-dist>>', target_dist)

    if which == 'breaks':
        dataset = dataset.map(sample)

    dataset = dataset.apply(tf.contrib.data.rejection_resample(
      class_func=lambda c: c,
      target_dist=target_dist))
    dataset = dataset.map(lambda a, _: a)

    return dataset.make_one_shot_iterator().get_next()

def run_thing(which):
    tf.reset_default_graph()
    with tf.Session() as sess:
        get_next = fix_sample_and_rebalance(which)
        returned = []
        for kk in range(0,100):
            try:
                sample=sess.run(get_next)
                returned.append(sample)
            except tf.errors.OutOfRangeError:
                break
    print(np.bincount(np.array(returned)))


In [121]:
run_thing('works')  # this works
run_thing('breaks')  # this doesn't

target-dist>> [0, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
[ 0 29  8 10 12 19 13  9]
target-dist>> [0, 0.25, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
[16 11 14 16 10 14 11  8]


In [119]:
# Assume your oversampling factors per class are fixed
# and you have 4 classes.
OVERSAMPLE_FACTOR = [1,1,1,1,1]
label= tf.cast(result.label, tf.int32)
images = tf.cast(result.uint8image, tf.int32)
# Now we need to reshape input image tensors to 4D, where the 
# first dimension is the image number in a batch of oversampled tensors.
label = tf.reshape(label,[1]) # so, (*,height,width,channels) in 4D

# Set up your predicates, which are 1D boolean tensors.
# Note you will have to squash the boolean tensors to 0-dimension.
# This seems illogical to me, but it is what it is.
pred0 = tf.reshape(tf.equal(label, tf.convert_to_tensor([0])), []) #0D tf.bool
pred1 = tf.reshape(tf.equal(label, tf.convert_to_tensor([1])), []) #0D tf.bool
pred2 = tf.reshape(tf.equal(label, tf.convert_to_tensor([2])), []) #0D tf.bool
pred3 = tf.reshape(tf.equal(label, tf.convert_to_tensor([3])), []) #0D tf.bool
pred4 = tf.reshape(tf.equal(label, tf.convert_to_tensor([4])), []) #0D tf.bool

# Build your callables (functions) that vertically stack an input image and
# label tensors X times depending on the accompanying oversample factor.
def f0(): return tf.concat( [images]*OVERSAMPLE_FACTOR[0],0), tf.concat([label]*OVERSAMPLE_FACTOR[0],0)
def f1(): return tf.concat( [images]*OVERSAMPLE_FACTOR[1],0), tf.concat([label]*OVERSAMPLE_FACTOR[1],0)
def f2(): return tf.concat( [images]*OVERSAMPLE_FACTOR[2],0), tf.concat([label]*OVERSAMPLE_FACTOR[2],0)
def f3(): return tf.concat( [images]*OVERSAMPLE_FACTOR[3],0), tf.concat([label]*OVERSAMPLE_FACTOR[3],0)
def f4(): return tf.concat( [images]*OVERSAMPLE_FACTOR[4],0), tf.concat([label]*OVERSAMPLE_FACTOR[4],0)

# Here we have N conditionals, one for each class.  These are exclusive
# but due to tf.case() not behaving every conditional gets evaluated.
[images, label] = tf.cond(pred0, f0, lambda: [images,label])
[images, label] = tf.cond(pred1, f1, lambda: [images,label])
[images, label] = tf.cond(pred2, f2, lambda: [images,label])
[images, label] = tf.cond(pred3, f3, lambda: [images,label])
[images, label] = tf.cond(pred3, f4, lambda: [images,label])

TypeError: Incompatible return types of true_fn and false_fn: The two structures don't have the same sequence type. First structure has type <type 'tuple'>, while second structure has type <type 'list'>.